In [ ]:
import pandas as pd

file_path = 'ZIF-8 data.xlsx'
xl = pd.ExcelFile(file_path)

threshold = 6e-8

with pd.ExcelWriter(file_path, engine="openpyxl", mode="a") as writer:
    for sheet_name in xl.sheet_names:
        df = xl.parse(sheet_name)
        
        column_pairs = [(df.columns[i], df.columns[i + 1]) for i in range(0, len(df.columns), 2)]
        print(f"Processing sheet: {sheet_name}, Identified column pairs: {column_pairs}")
        
        for pair_idx, (time_col, current_col) in enumerate(column_pairs):
            df[current_col] = pd.to_numeric(df[current_col], errors='coerce')
            
            pulse_onsets = []
            for i in range(1, len(df)):
                if df[current_col].iloc[i] >= threshold and df[current_col].iloc[i - 1] < threshold:
                    pulse_onsets.append((df[time_col].iloc[i - 1], df[current_col].iloc[i - 1]))
            
            if not pulse_onsets:
                print(f"No pulses detected for {time_col} & {current_col}, skipping.")
                continue
            
            normalized_data = pd.DataFrame()
            for j in range(len(pulse_onsets)):
                if j < len(pulse_onsets) - 1:
                    pulse_data = df[(df[time_col] >= pulse_onsets[j][0]) & (df[time_col] < pulse_onsets[j+1][0])]
                else:
                    pulse_data = df[(df[time_col] >= pulse_onsets[j][0])]
                
                relative_time = pulse_data[time_col] - pulse_data[time_col].iloc[0]
                
                pulse_data = pulse_data[relative_time < 20.1]
                relative_time = relative_time[relative_time < 20.1]
                
                max_pulse_current = pulse_data[current_col].max()
                normalized_values = pulse_data[current_col] / max_pulse_current
                
                temp_df = pd.DataFrame({
                    f'Total Time {j+1}': pulse_data[time_col].reset_index(drop=True),
                    f'Relative Time {j+1}': relative_time.reset_index(drop=True),
                    f'Normalized Ion Current {j+1}': normalized_values.reset_index(drop=True),
                    f'Ion Current {j+1}': pulse_data[current_col].reset_index(drop=True)
                })
                
                normalized_data = pd.concat([normalized_data, temp_df], axis=1)
            
            sheet_output_name = f"Proc_{sheet_name[:15]}_{pair_idx+1}"[:31]
            
            if not normalized_data.empty:
                normalized_data.to_excel(writer, sheet_name=sheet_output_name, index=False)
                print(f"Processed data saved in sheet '{sheet_output_name}' for {sheet_name}, columns {time_col} & {current_col}.")
            else:
                print(f"Skipping empty sheet '{sheet_output_name}'.")

print("Processing complete.")